In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# 各パス指定

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'

# 分類数設定

In [3]:
NUM_CLASSES = 5

# 学習データ読み込み

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

In [9]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# モデル訓練

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
29/31 [===========================>..] - ETA: 0s - loss: 1.6254 - accuracy: 0.2524
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 2s 16ms/step - loss: 1.6208 - accuracy: 0.2554 - val_loss: 1.5225 - val_accuracy: 0.3772
Epoch 2/1000
30/31 [============================>.] - ETA: 0s - loss: 1.5296 - accuracy: 0.3276
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 11ms/step - loss: 1.5282 - accuracy: 0.3288 - val_loss: 1.4282 - val_accuracy: 0.4627
Epoch 3/1000
29/31 [===========================>..] - ETA: 0s - loss: 1.4420 - accuracy: 0.3823
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 7ms/step - loss: 1.4412 - accuracy: 0.3817 - val_loss: 1.3384 - val_accuracy: 0.4973
Epoch 4/1000
27/31 [=========================>....] - ETA: 0s - loss: 1.3771 - accuracy: 0.4204

29/31 [===========================>..] - ETA: 0s - loss: 0.7863 - accuracy: 0.6886
Epoch 28: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 7ms/step - loss: 0.7842 - accuracy: 0.6905 - val_loss: 0.4960 - val_accuracy: 0.9145
Epoch 29/1000
31/31 [==============================] - ETA: 0s - loss: 0.7807 - accuracy: 0.6907
Epoch 29: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 7ms/step - loss: 0.7807 - accuracy: 0.6907 - val_loss: 0.4904 - val_accuracy: 0.9184
Epoch 30/1000
27/31 [=========================>....] - ETA: 0s - loss: 0.7773 - accuracy: 0.6889
Epoch 30: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 8ms/step - loss: 0.7755 - accuracy: 0.6917 - val_loss: 0.4861 - val_accuracy: 0.9145
Epoch 31/1000
22/31 [====================>.........] - ETA: 0s - loss: 0.7746 - accuracy: 0.6946
Epoch 31

17/31 [===============>..............] - ETA: 0s - loss: 0.6771 - accuracy: 0.7463
Epoch 55: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 6ms/step - loss: 0.6634 - accuracy: 0.7531 - val_loss: 0.3489 - val_accuracy: 0.9330
Epoch 56/1000
19/31 [=================>............] - ETA: 0s - loss: 0.6663 - accuracy: 0.7459
Epoch 56: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 6ms/step - loss: 0.6676 - accuracy: 0.7454 - val_loss: 0.3462 - val_accuracy: 0.9338
Epoch 57/1000
18/31 [================>.............] - ETA: 0s - loss: 0.6530 - accuracy: 0.7574
Epoch 57: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 7ms/step - loss: 0.6590 - accuracy: 0.7533 - val_loss: 0.3404 - val_accuracy: 0.9376
Epoch 58/1000
31/31 [==============================] - ETA: 0s - loss: 0.6658 - accuracy: 0.7510
Epoch 58

26/31 [========================>.....] - ETA: 0s - loss: 0.6188 - accuracy: 0.7626
Epoch 82: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 10ms/step - loss: 0.6091 - accuracy: 0.7659 - val_loss: 0.2847 - val_accuracy: 0.9384
Epoch 83/1000
20/31 [==================>...........] - ETA: 0s - loss: 0.6496 - accuracy: 0.7629
Epoch 83: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 10ms/step - loss: 0.6272 - accuracy: 0.7698 - val_loss: 0.2862 - val_accuracy: 0.9338
Epoch 84/1000
22/31 [====================>.........] - ETA: 0s - loss: 0.6185 - accuracy: 0.7713
Epoch 84: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 9ms/step - loss: 0.6181 - accuracy: 0.7687 - val_loss: 0.2796 - val_accuracy: 0.9392
Epoch 85/1000
22/31 [====================>.........] - ETA: 0s - loss: 0.6066 - accuracy: 0.7702
Epoch 

Epoch 109/1000
25/31 [=======================>......] - ETA: 0s - loss: 0.5570 - accuracy: 0.7950
Epoch 109: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 13ms/step - loss: 0.5670 - accuracy: 0.7906 - val_loss: 0.2613 - val_accuracy: 0.9415
Epoch 110/1000
24/31 [======================>.......] - ETA: 0s - loss: 0.5858 - accuracy: 0.7881
Epoch 110: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 13ms/step - loss: 0.5749 - accuracy: 0.7936 - val_loss: 0.2586 - val_accuracy: 0.9461
Epoch 111/1000
28/31 [==========================>...] - ETA: 0s - loss: 0.5667 - accuracy: 0.7857
Epoch 111: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 14ms/step - loss: 0.5686 - accuracy: 0.7844 - val_loss: 0.2569 - val_accuracy: 0.9423
Epoch 112/1000
28/31 [==========================>...] - ETA: 0s - loss: 0.5808 - a

Epoch 136/1000
27/31 [=========================>....] - ETA: 0s - loss: 0.5560 - accuracy: 0.7934
Epoch 136: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 0s 15ms/step - loss: 0.5547 - accuracy: 0.7934 - val_loss: 0.2452 - val_accuracy: 0.9423
Epoch 137/1000
27/31 [=========================>....] - ETA: 0s - loss: 0.5586 - accuracy: 0.7960
Epoch 137: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 1s 19ms/step - loss: 0.5549 - accuracy: 0.7970 - val_loss: 0.2426 - val_accuracy: 0.9400
Epoch 138/1000
28/31 [==========================>...] - ETA: 0s - loss: 0.5618 - accuracy: 0.7874
Epoch 138: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 1s 19ms/step - loss: 0.5628 - accuracy: 0.7867 - val_loss: 0.2421 - val_accuracy: 0.9438
Epoch 139/1000
26/31 [========================>.....] - ETA: 0s - loss: 0.5531 - a

Epoch 163/1000
29/31 [===========================>..] - ETA: 0s - loss: 0.5500 - accuracy: 0.7974
Epoch 163: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 1s 21ms/step - loss: 0.5525 - accuracy: 0.7972 - val_loss: 0.2346 - val_accuracy: 0.9438
Epoch 164/1000
30/31 [============================>.] - ETA: 0s - loss: 0.5523 - accuracy: 0.7966
Epoch 164: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 1s 18ms/step - loss: 0.5504 - accuracy: 0.7972 - val_loss: 0.2316 - val_accuracy: 0.9461
Epoch 165/1000
31/31 [==============================] - ETA: 0s - loss: 0.5508 - accuracy: 0.7952
Epoch 165: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
31/31 [==============================] - 1s 26ms/step - loss: 0.5508 - accuracy: 0.7952 - val_loss: 0.2339 - val_accuracy: 0.9453
Epoch 166/1000
29/31 [===========================>..] - ETA: 0s - loss: 0.5432 - a

Epoch 190/1000
22/31 [====================>.........] - ETA: 0s - loss: 0.5248 - accuracy: 0.8129

In [ ]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

In [ ]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# 混同行列

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

# Tensorflow-Lite用のモデルへ変換

In [ ]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [ ]:
# モデルを変換(量子化)
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# 推論テスト

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# 推論実施
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))